# __Brough Lecture Notes: GARCH Models - Asymmetric Models__

<br>

Finance 5330: Financial Econometrics <br>
Tyler J. Brough <br>
Last Updated: March 28, 2019 <br>
<br>
<br>

These notes are based in part on several chapters in the series of books on [Market Risk Analysis](http://carolalexander.org/market-risk-analysis/) by Carol Alexander.

<br>

There is a mind-numbing alphabet soup of variations on GARCH models. These notes introduce you to one simple extension known as the GJR-GARCH model, named for the initials of its [inventors](https://bit.ly/2U1d8Kj).

<br>

Ever since [Black 1976](https://bit.ly/2CGBNck) economists have become aware of what Black called the _leverage effect_.
This basically means that there is an _asymmetric_ response to past volatility shocks.

<br>

To account for this GJR add a single extra _leverage_ parameter that augments the weight of the response in the model to negative market shocks. 

<br>

The model can be written down as: 

<br>

$$
\sigma_{t}^{2} = \omega + \alpha \epsilon_{t-1}^{2} + \lambda I_{\{\epsilon_{t-1} < 0\}} \epsilon_{t-1}^{2} + \beta \sigma_{t-1}^{2}
$$

<br>

where $I_{\{\epsilon_{t-1} < 0 \}} = 1$ if $\epsilon_{t-1} < 0$ and $0$ otherwise (called an indicator function).

<br>

MLE parameter estimation is based on the usual GARCH likelihood function, where $\sigma_{t}$ depends now on $\lambda$.

<br>

__NB:__ Note that if $\lambda = 0$ then we have a simple GARCH(1,1) model, so the GJR-GARCH model is a generalization
        of the standard GARCH model. 
        
<br>

__NB:__ Note that a general GJR-GARCH(p, o, q) model could be written down similar to a GARCH(p, q) model where the $o$
        parameter is the number of lags in the _leverage term_. 
        
<br>

The GJR-GARCH model has often been found to improve the fit to data relative to a standard symmetric GARCH model. 

<br>

The standard model specification is a GRJ-GARCH(1,1,1) model (similar to the GARCH(1,1) case). 

## Estimation via MLE

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]

import seaborn
from numpy import size, log, exp, pi, sum, diff, array, zeros, diag, mat, asarray, sqrt, copy
from numpy.linalg import inv

from scipy.optimize import fmin_slsqp, minimize

In [27]:
def gjr_garch_likelihood(parameters, data, sigma2, out=None):
    '''Returns negative log-likelihood for GJR-GARCH(1,1,1) model.'''
    mu = parameters[0]
    omega = parameters[1]
    alpha = parameters[2]
    gamma = parameters[3]
    beta = parameters[4]
    
    T = size(data,0)
    eps = data - mu
    # Data and sigma2 are T by 1 vectors
    for t in range(1,T):
        sigma2[t] = (omega + alpha * eps[t-1]**2 + gamma * eps[t-1]**2 * (eps[t-1]<0) + beta * sigma2[t-1])
        
    logliks = 0.5*(log(2*pi) + log(sigma2) + eps**2/sigma2)
    loglik = sum(logliks)
    
    if out is None:
        return loglik
    else:
        return loglik, logliks, copy(sigma2)

In [28]:
def gjr_constraint(parameters, data, sigma2, out=None):
    '''Constraint that alpha+gamma/2+beta<=1'''
    
    alpha = parameters[2]
    gamma = parameters[3]
    beta = parameters[4]
    
    return array([1-alpha-gamma/2-beta])

In [29]:
def hessian_2sided(fun, theta, args):
    f = fun(theta, *args)
    h = 1e-5*np.abs(theta)
    thetah = theta + h
    h = thetah - theta
    K = size(theta,0)
    h = np.diag(h)
    
    fp = zeros(K)
    fm = zeros(K)
    for i in range(K):
        fp[i] = fun(theta+h[i], *args)
        fm[i] = fun(theta-h[i], *args)
        
        
    fpp = zeros((K,K))
    fmm = zeros((K,K))
    for i in range(K):
        for j in range(i,K):
            fpp[i,j] = fun(theta + h[i] + h[j], *args)
            fpp[j,i] = fpp[i,j]
            fmm[i,j] = fun(theta - h[i] - h[j], *args)
            fmm[j,i] = fmm[i,j]
            
    hh = (diag(h))
    hh = hh.reshape((K,1))
    hh = hh @ hh.T
    
    H = zeros((K,K))
    
    for i in range(K):
        for j in range(i,K):
            H[i,j] = (fpp[i,j] - fp[i] - fp[j] + f + f - fm[i] - fm[j] + fmm[i,j])/hh[i,j]/2
            H[j,i] = H[i,j]
            
    return H

In [30]:
ibm = pd.read_csv("./data/IBM-1999-2003.csv", parse_dates=True, index_col=1)

In [31]:
ibm.head()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
1999-01-04,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,183.0000,-0.007458,2,-0.007458,-0.000919
1999-01-05,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,189.6250,0.036202,2,0.036202,0.013582
1999-01-06,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,188.7500,-0.004614,2,-0.004614,0.022140
1999-01-07,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,190.1875,0.007616,2,0.007616,-0.002051
1999-01-08,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,187.5625,-0.013802,2,-0.013802,0.004221


In [32]:
ibm.tail()

,PERMNO,TICKER,COMNAM,PERMCO,PRC,RET,CFACPR,RETX,sprtrn
date,,,,,,,,,
2003-12-24,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.27,-0.005604,1,-0.005604,-0.001807
2003-12-26,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.90,0.006828,1,0.006828,0.001691
2003-12-29,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,93.52,0.006674,1,0.006674,0.012401
2003-12-30,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.63,-0.009517,1,-0.009517,0.000144
2003-12-31,12490,IBM,INTERNATIONAL BUSINESS MACHS COR,20990,92.68,0.000540,1,0.000540,0.002055


In [33]:
prc = np.abs(ibm.PRC.values)
ret = 100 * ibm.RET.values

In [34]:
ret.mean()

0.03420429936305733

In [35]:
ret.var()

6.305847511637566

In [36]:
startingVals = array([ret.mean(), ret.var() * .01, 0.03, 0.09, .09])

In [37]:
finfo = np.finfo(np.float64)
bounds = [(-10*ret.mean(), 10*ret.mean()), (finfo.eps, 2*ret.var()), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0)]

In [38]:
T = ret.shape[0]

In [39]:
sigma2 = np.ones(T) * ret.var()
args = (np.asarray(ret), sigma2)

In [68]:
estimates = fmin_slsqp(gjr_garch_likelihood, startingVals, f_ieqcons=gjr_constraint, bounds=bounds, args = args)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2839.0106169167507
            Iterations: 28
            Function evaluations: 216
            Gradient evaluations: 28


In [69]:
estimates

array([0.02951199, 0.018095  , 0.00337605, 0.08269122, 0.95527834])

In [72]:
bob = minimize(fun=gjr_garch_likelihood, x0=startingVals, args=args, method='Nelder-Mead') #, bounds=bounds)

/home/brough/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log
  from ipykernel import kernelapp as app


In [73]:
bob

 final_simplex: (array([[0.02113193, 0.01216813, 0.00318826, 0.08324996, 0.95705696],
       [0.02109723, 0.01216096, 0.00318452, 0.08322738, 0.95707071],
       [0.02119908, 0.01213519, 0.00318713, 0.0832479 , 0.95706285],
       [0.02115491, 0.01215956, 0.00318295, 0.08325718, 0.95705829],
       [0.02118094, 0.01217498, 0.00318836, 0.08321967, 0.95706331],
       [0.02107593, 0.01216487, 0.00319355, 0.08323139, 0.95706008]]), array([2838.87291525, 2838.8729159 , 2838.8729161 , 2838.87291614,
       2838.87291706, 2838.87291715]))
           fun: 2838.8729152505502
       message: 'Optimization terminated successfully.'
          nfev: 929
           nit: 578
        status: 0
       success: True
             x: array([0.02113193, 0.01216813, 0.00318826, 0.08324996, 0.95705696])